In [1]:
# library
import torch
import numpy as np
import pandas as pd
from torchmetrics import R2Score

r2score = R2Score()

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# Model
class MonotonicLinear(torch.nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.input_size  = input_size
    self.output_size = output_size
    self.weights = torch.nn.Parameter(torch.rand((output_size, input_size), dtype=torch.float32))

  def forward(self, x):
    z = torch.mm(x, torch.exp(self.weights.t()))
    return z

class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.input11 = MonotonicLinear(1, 4)
    self.input12 = torch.nn.Linear(2, 4)
    self.max1    = torch.nn.MaxPool1d(4)

    self.input21 = MonotonicLinear(1, 4)
    self.input22 = torch.nn.Linear(2, 4)
    self.max2    = torch.nn.MaxPool1d(4)

    self.output = torch.nn.MaxPool1d(2)

  def forward(self, x_m, x_u):
    z1 = torch.add(self.input11(x_m), self.input12(x_u))
    z1 = self.max1(z1)

    z2 = torch.add(self.input21(x_m), self.input22(x_u))
    z2 = self.max1(z2)

    z = torch.concat((z1,z2), axis=1)
    z = self.output(-z)

    return -z

In [3]:
# Model evaluation
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x_m_data, testset.x_u_data)
    
    r2 = r2score(pred_Y, testset.y_data)
    return r2.item()

In [4]:
# Data class
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+1: end+1]
    Tz_k   = np.array(df['room_temp']).reshape(-1,1)[start: end]
    msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[start+1: end+1]

    tmp_x_u = np.concatenate((Tz_k, msa_k1), axis=1)
    tmp_y   = np.array(df['room_temp']).reshape(-1,1)[start+1: end+1]

    self.x_m_data = torch.tensor(Tsa_k1, dtype=torch.float32)
    self.x_u_data = torch.tensor(tmp_x_u, dtype=torch.float32)
    self.y_data   = torch.tensor(tmp_y, dtype=torch.float32)

  def __len__(self):
    return len(self.y_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp_m  = self.x_m_data[idx]
    inp_u  = self.x_u_data[idx]
    outp   = self.y_data[idx]
    sample = {'inp_m':inp_m, 'inp_u':inp_u, 'outp':outp}
    return sample

In [5]:
# Early stopping function
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.00001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    R2_test    = np.array([])
    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)
    for _ in range(0, max_epochs+1):
        net.train()
        for (_, batch) in enumerate(train_ldr):
            X_m = batch['inp_m']
            X_u = batch['inp_u']
            Y   = batch['outp']

            optimizer.zero_grad()
            output = net(X_m, X_u)
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X_u[:,0].reshape(-1,1))
            loss_val.backward()
            optimizer.step()

        net.eval()
        R2_test = np.append(R2_test, eval(net, test_ds))
        
        if(early_stop(list = R2_test, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_test

In [7]:
# main
df_result = pd.DataFrame({'n_train':[], 'smooth':[], 'lr':[], 'best_epoch':[], 'R2':[]})
for _n_train in [16, 32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for _lr in [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5]:

            # Create network
            device = torch.device("cpu")
            net = Net().to(device)

            # Create Dataset and DataLoader objects
            src_file = 'C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv'
            n_train  = _n_train
            train_ds = Data(src_file, start=0,       end=n_train)
            test_ds  = Data(src_file, start=n_train, end=1600)

            # train
            R2_test = train(net, train_ds, test_ds, lr=_lr, min_epochs=1000, max_epochs=100000, patience=500, smooth=_smooth)

            # results
            df_result.loc[len(df_result)] = [_n_train, _smooth, _lr, np.argmax(R2_test), np.max(R2_test)]

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)

    n_train  smooth      lr  best_epoch          R2
0      16.0   0.000  0.0001         3.0 -34958.1523
1      16.0   0.000  0.0010         3.0 -10384.9629
2      16.0   0.000  0.0100         3.0 -22169.9590
3      16.0   0.000  0.1000         3.0   -812.5502
4      16.0   0.000  0.2000         1.0   -579.7729
5      16.0   0.000  0.5000         1.0  -1344.0801
6      16.0   0.001  0.0001         3.0 -50409.7852
7      16.0   0.001  0.0010         3.0  -6091.3198
8      16.0   0.001  0.0100         3.0 -34343.2891
9      16.0   0.001  0.1000         1.0   -243.4563
10     16.0   0.001  0.2000         3.0  -1299.3234
11     16.0   0.001  0.5000         2.0  -1865.8385
12     16.0   0.010  0.0001         3.0  -1968.1769
13     16.0   0.010  0.0010         0.0   -162.3411
14     16.0   0.010  0.0100         3.0 -44349.7852
15     16.0   0.010  0.1000         0.0   -165.1587
16     16.0   0.010  0.2000         3.0    -54.1379
17     16.0   0.010  0.5000         3.0   -318.4189
18     32.0 